# MNIST 데이터 이해

![MNIST](../img/mnist1.png)
- MNIST : 숫자 0부터 9까지의 이미지로 구성된 손글씨 데이터셋이다. 총 6만개의 훈련 데이터와 레이블, 총 1만개의 테스트 데이터와 레이블로 구성되어 있다.
- MNIST 문제는 손글씨로 적힌 숫자 이미지가 무슨 숫자인지 맞추는 문제이다.

![MNIST](../img/mnist2.png)
- 위 문제를 풀기 위해서는 총 784(28*28픽셀)의 원소를 가진 벡터로 만든다. 총 784개의 특성을 가진 샘플이 된다.
```py
for X, Y in data_loader:
    # 입력 이미지를 [batch_size × 784]의 크기로 reshape
    # 레이블은 원-핫 인코딩
    X = X.view(-1, 28*28)
```

# torchvision(토치비전)

- torchvision : 유명 데이터셋, 구현된 모델, 일반적인 이미지 전처리 도구들을 포함한 패키지이다. (자연어 처리에 사용되는 torchtext라는 패키지도 존재한다.)

# 분류기 구현 사전 설정

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

In [3]:
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: cpu


- Google Colab에서 사용시 런타임>런타임 유형 변경>하드웨어 가속기>GPU 를 선택하여 GPU 연산을 실행할 수 있다.

In [4]:
# 랜덤 시드 고정
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
# 하이퍼파라미터 변수로 생성
training_epochs = 15
batch_size = 100

# MNIST 분류기 구현

In [6]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

9913344it [04:51, 34017.82it/s]


Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



29696it [00:00, 322978.44it/s]           


Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



1649664it [00:01, 1041301.66it/s]                             


Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



5120it [00:00, 5048151.50it/s]          
C:\Users\hcw2l\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw

Processing...
Done!


- root : 데이터를 다운로드 받을 경로
- train : True의 경우 훈련 데이터를 리턴 받으며, False의 경우 테스트 데이터를 리턴 받는다.
- transform : 현재 데이터를 파이토치 텐서로 변환한다.
- download : 해당 경로에 MNIST 데이터가 없을 경우 다운로드 한다.

In [7]:
# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                        batch_size=batch_size, # 배치 크기 = 100
                        shuffle=True,
                        drop_last=True)

- dataset : 로드할 대상을 의미한다.
- batch_size : 배치 크기를 의미한다.
- shuffle : epoch마다 미니 배치 셔플 여부를 의미한다.
- drop_last : 마지막 배치를 버릴 것인지 정한다.
    - dopr_last를 실행하는 이유 : 마지막 배치가 다른 미니 배치보다 개수가 적을 경우 마지막 배치가 경사 하강법에 사용될 때 상대적으로 과대 평가되는 현상을 막기 위함이다.

In [8]:
# MNIST data image of shape 28 * 28 = 784
linear = nn.Linear(784, 10, bias=True).to(device)

- to() : 연산을 어디서 수행할지 결정한다. (GPU를 사용할 경우 적어줘야 한다.)
- bias : 편향 b의 사용 여부를 나타낸다.

In [9]:
# 비용 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss().to(device)
# torch.nn.functional.cross_entropy(), torch.nn.CrossEntropyLoss() 둘 다 내부적으로 소프트맥스 함수 포함
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [10]:
for epoch in range(training_epochs): # 15
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader:
        # 배치 크기가 100이므로 아래의 연산에서 X는 (100, 784)의 텐서가 된다.
        X = X.view(-1, 28 * 28).to(device)
        # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수.
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.534912527
Epoch: 0002 cost = 0.359308630
Epoch: 0003 cost = 0.331088215
Epoch: 0004 cost = 0.316574216


In [ ]:
# 테스트 데이터를 사용하여 모델을 테스트한다.
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # MNIST 테스트 데이터에서 무작위로 하나를 뽑아서 예측을 해본다
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

    plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
    plt.show()